<a href="https://colab.research.google.com/github/Minwoo-study/Korean_Classic_DB/blob/main/%ED%95%9C%EA%B5%AD%EA%B3%A0%EC%A0%84%EC%A2%85%ED%95%A9DB_%EC%9B%90%EB%AC%B8_%EB%8B%A8%EC%96%B4_%EB%8B%A8%EC%9C%84_tokenizing%2Bword2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국고전종합DB 원문 번역문 word2vec

In [2]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip install -U kiwipiepy gensim kss -q

In [3]:
import pandas as pd
# import pickle5 as pickle
import pickle
import re
import regex
from tqdm import tqdm
tqdm.pandas()

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from kss import split_sentences
import itertools
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib as mpl
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
import matplotlib.colors as clr

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### 원문 Word2vec

In [4]:
#분석용 피클파일 불러오기
# https://drive.google.com/file/d/1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1BCAZmBi3OK9GFtC2prab4wCRQtJaYUVq" -o df_sent.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1533      0 --:--:-- --:--:-- --:--:--  1533
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  674M  100  674M    0     0   117M      0  0:00:05  0:00:05 --:--:--  142M


In [5]:
with open('df_sent.pkl', "rb") as fh:
  df_sent = pickle.load(fh)

In [6]:
df_sent

,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"
...,...,...,...,...,...
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",人心世道日趍於汙,"[人, 心, 世, 道, 日, 趍, 於, 汙]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",不知問學之爲何名而義理之爲何物,"[不, 知, 問, 學, 之, 爲, 何, 名, 而, 義, 理, 之, 爲, 何, 物]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",如先生者更於何叫起來也,"[如, 先, 生, 者, 更, 於, 何, 叫, 起, 來, 也]"
105623,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",只得拊遺卷爲之太息而已,"[只, 得, 拊, 遺, 卷, 爲, 之, 太, 息, 而, 已]"


In [7]:
df_sent = df_sent.reset_index(drop= True)

### 원문에서 같이 보고 싶은 단어 정리

#### 단어가 하나일 때

In [123]:
# # words(token 열의 각 값(리스트형식), 합치고 싶은 단어)

# def words (lists, word):
#   global result
#   result =[]
#   char1 , char2 = list(word)
#   for index, value in enumerate(lists):
#     try: 
#       if (value == char1) & (lists[index+1]==char2):
        
#         result.append(word)
#         print(word)
#       elif (value ==char2)&(lists[index-1]==char1) :
#         continue
#       else :
#         result.append(value)
#     except IndexError :
#       result.append(value)
#       continue
  


In [8]:
# words(token 열의 각 값(리스트형식), 합치고 싶은 단어)
def words (lists, word):
  global result
  char1 , char2 = list(word)
  for index, value in enumerate(lists):
    try: 
      if (value == char1) & (lists[index+1]==char2):
        lists[index] = word
        print(word)
      elif (value ==char2)&(lists[index-1]==word) :
        del lists[index]
      else :
        continue
    except IndexError :
      result =lists

In [177]:
df_sent

,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
1,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
3,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
4,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"
...,...,...,...,...,...
3796187,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",人心世道日趍於汙,"[人, 心, 世, 道, 日, 趍, 於, 汙]"
3796188,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",不知問學之爲何名而義理之爲何物,"[不, 知, 問, 學, 之, 爲, 何, 名, 而, 義, 理, 之, 爲, 何, 物]"
3796189,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",如先生者更於何叫起來也,"[如, 先, 生, 者, 更, 於, 何, 叫, 起, 來, 也]"
3796190,ITKC_MO_1252A_0290_000_0030,嗚呼。此吾響山公遺文也。公大節旣昭然如日星之在天。其不朽何待乎文哉。獨後死者之論。以爲公歿而...,"\n오호라, 이 책은 우리 향산공(響山公)의 유문(遺文)이다. 공의 큰 절의가 이미...",只得拊遺卷爲之太息而已,"[只, 得, 拊, 遺, 卷, 爲, 之, 太, 息, 而, 已]"


In [9]:
# 전체 데이터에서 진행하기
for i in tqdm(df_sent.index):
  words(df_sent['token'][i], '我國') # 문자 단위로 토크나이징 된 리스트들이 들어있는 열과 합치고 싶은 단어를 넣기
  #print(result)

df_sent.head()

  1%|          | 20169/3796192 [00:00<00:38, 97794.01it/s] 

我國
我國
我國


  1%|▏         | 49987/3796192 [00:00<00:39, 94063.76it/s]

我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國
我國


  2%|▏         | 69396/3796192 [00:00<00:39, 94979.35it/s]

...

100%|█████████▉| 3791173/3796192 [00:49<00:00, 29502.57it/s]

我國
我國
我國


100%|██████████| 3796192/3796192 [00:49<00:00, 76652.69it/s]

我國
我國


,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
1,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
3,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
4,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"


In [10]:
# 단어 
word = '我國'
inword = [i for i in tqdm(df_sent.index) if word in df_sent['token'][i]]
print(inword)

100%|██████████| 3796192/3796192 [00:25<00:00, 147514.76it/s]

[5204, 13700, 22669, 32373, 34945, 38525, 38818, 38962, 39050, 39278, 42186, 42258, 42276, 42424, 42425, 42532, 42544, 42560, 42577, 42599, 42712, 42722, 42824, 42826, 42951, 42957, 42978, 42985, 42996, 43007, 43024, 43073, 43162, 43208, 43219, 43248, 43267, 43364, 43386, 43420, 47504, 48721, 48984, 52619, 52629, 54682, 56295, 57124, 57552, 60086, 60844, 60872, 61291, 66972, 75286, 75385, 88303, 100376, 111735, 114114, 136857, 139116, 139411, 139672, 142980, 143463, 151306, 153992, 169833, 171199, 184044, 190636, 192091, 192176, 192184, 195678, 195783, 196001, 196021, 196025, 196411, 196822, 197323, 197426, 197435, 197535, 200098, 200437, 202065, 203679, 204753, 204826, 204835, 205352, 205383, 205427, 205626, 205680, 206159, 206297, 206828, 207627, 208827, 211155, 217044, 224077, 224331, 226037, 226129, 226206, 226682, 232338, 233163, 239697, 241071, 241430, 241716, 241926, 242098, 242823, 243410, 244182, 244442, 245686, 277488, 277661, 279234, 280732, 282236, 282323, 282333, 282611, 2

In [11]:
#단어 포함된 데이터 프레임 확인
df_sent.loc[inword]

,ID,원문,번역문,sentence,token
5204,ITKC_MO_0001A_0120_010_0010,廣明二年七月八日。諸道都統檢校太尉某。告黃巢。夫守正修常曰道。臨危制變曰權。智者成之於順時。...,\n광명(廣明) 2년(881) 7월 8일에 제도도통 검교태위(諸道都統檢校太尉) 모(...,比者我國家德深含垢,"[比, 者, 我國, 家, 德, 深, 含, 垢]"
13700,ITKC_MO_0002A_0030_010_0180,靑鶴洞碑。鄭弘溟銘曰。若高麗，百濟，新羅。國雖一域。粤有蓬萊，瀛洲，方丈。山則三神。積氣扶桑...,"\n고려와 백제와 신라로 말하면, 나라는 비록 한 지역에 속한다고 하겠지만 봉래(蓬...",惟我國家,"[惟, 我國, 家]"
22669,ITKC_MO_0004A_0050_010_0010,世多說東明王神異之事。雖愚夫騃婦。亦頗能說其事。僕嘗聞之。笑曰。先師仲尼。不語怪力亂神。此實...,\n세상에서 동명왕(東明王)의 신통하고 이상한 일을 많이 말한다. 비록 어리석은 남...,欲使夫天下知我國本聖人之都耳,"[欲, 使, 夫, 天, 下, 知, 我國, 本, 聖, 人, 之, 都, 耳]"
32373,ITKC_MO_0004A_0210_010_0010,兒郞偉。有屋曰廩。實惟穀藏之源。在天爲星。蓋是庫樓之象。於國莫大。其制宜宏。我國家宅萬世之都...,"\n아랑위(兒郞偉), 집에 있어서는 곳집이라 하니 실상 곡식을 저장하는 근원이요, ...",我國家宅萬世之都,"[我國, 家, 宅, 萬, 世, 之, 都]"
34945,ITKC_MO_0004A_0230_020_0110,吳德全爲詩。遒邁勁俊。其詩之膾炙人口者。不爲不多。然未見能押強韻。儼若天成者。及於北山欲題戟...,\n오덕전은 시를 힘차고 준수하게 지었다. 그 시가 사람들의 입에 오르내린 것이 퍽...,我國人茫然無以對,"[我國, 人, 茫, 然, 無, 以, 對]"
...,...,...,...,...,...
3787514,ITKC_MO_1252A_0240_010_0150,宣城搴芝山南遠岱之北。有所謂招魂葬者。卽贈軍器寺僉正琴公海龍之墓也。公奉化人。高麗平章事儀之...,\n선성(宣城 예안(禮安)의 고호)의 건지산(搴芝山) 남쪽 원대(遠岱)의 북쪽에 초...,我國外亂,"[我國, 外, 亂]"
3791962,ITKC_MO_1252A_0270_000_0010,公諱晩燾字觀必號響山。又曰直齋。姓李氏眞城人。麗末以縣吏中司馬諱碩其始祖。六傳而至文純公諱滉...,"\n공은, 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 또는 직재(直齋...",而竊伏念我國與日本,"[而, 竊, 伏, 念, 我國, 與, 日, 本]"
3793265,ITKC_MO_1252A_0270_000_0030,惟我國家隆盛。退陶夫子以道學啓一治文明之運。國家將終。響山先生以節義扶萬世綱常之重。盖道學節...,\n우리나라가 융성하였을 때 퇴도(退陶) 선생이 도학으로써 일치문명(一治文明)의 운...,惟我國家隆盛,"[惟, 我國, 家, 隆, 盛]"
3794536,ITKC_MO_1252A_0280_000_0010,府君諱晩燾字觀必。號響山。又曰直齋。姓李氏。始祖諱碩。麗末以眞寶縣吏中司馬。生諱子脩。登第討...,"\n부군은 휘는 만도(晩燾), 자는 관필(觀必), 호는 향산(響山) 혹은 직제(直齋...",而竊伏念我國與日本壬辰之後,"[而, 竊, 伏, 念, 我國, 與, 日, 本, 壬, 辰, 之, 後]"


#### 단어 여러개 한번에 합치기



In [140]:
# def word_lists (lists, word_list):
#   global result
#   result =[]
#   for word in word_list :
#     char1 , char2 = list(word)
#     for index, value in enumerate(lists):
#       try: 
#         if (value == char1) & (lists[index+1]==char2):
#           #test[index] = '南地'
#           #del test[index+1]
#           result.append(word)
#           print(word)
#         elif (value ==char2)&(lists[index-1]==char1) :
#           continue
#         else :
#           result.append(value)
#       except IndexError :
#         result.append(value)
#         continue

In [12]:
def word_lists (lists, word_list):
  global result
  
  for word in word_list :
    char1 , char2 = list(word)
    for index, value in enumerate(lists):
      try: 
        if (value == char1) & (lists[index+1]==char2):
          lists[index] = word
          print(word)
        elif (value ==char2)&(lists[index-1]==word) :
          del lists[index]
        else :
          continue
      except IndexError :
        result =lists

In [13]:
word_list = ['我國', '天下']  #단어 단위로 토크나이징할 단어 목록 작성
for i in tqdm(df_sent.index):
  word_lists(df_sent['token'][i], word_list)

df_sent.head()

  0%|          | 14297/3796192 [00:00<00:52, 71496.90it/s]

天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下


  1%|          | 29101/3796192 [00:00<00:51, 73238.24it/s]

天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下


  1%|          | 43689/3796192 [00:00<00:52, 71536.27it/s]

...

100%|█████████▉| 3789800/3796192 [00:59<00:00, 26762.53it/s]

天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下
天下


100%|██████████| 3796192/3796192 [00:59<00:00, 63453.62it/s]

天下
天下
天下
天下
天下
天下
天下
天下


,ID,원문,번역문,sentence,token
0,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,記有之,"[記, 有, 之]"
1,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,曰酒醴之美,"[曰, 酒, 醴, 之, 美]"
2,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,而玄酒明水之尙,"[而, 玄, 酒, 明, 水, 之, 尙]"
3,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,貴五味之本也,"[貴, 五, 味, 之, 本, 也]"
4,ITKC_MO_0001A_0010_010_0010,記有之。曰酒醴之美。而玄酒明水之尙。貴五味之本也。黼黻文繡之美。而疏布之尙。反女功之始也。古...,\n《예기》에 이르기를 “단술이 맛이 좋긴 하지만 제사 때에 현주와 명수 같은 물을...,黼黻文繡之美,"[黼, 黻, 文, 繡, 之, 美]"


In [15]:
#확인 
word1 = '我國'
word2 ='天下'
inword = [i for i in tqdm(df_sent.index) if (word1 in df_sent['token'][i])&(word2 in df_sent['token'][i])]
print(inword)


100%|██████████| 3796192/3796192 [00:57<00:00, 66574.13it/s]

[22669, 1200513, 1421620, 1537458, 1964926, 2252863]


In [16]:
df_sent['token'][22669]

['欲', '使', '夫', '天下', '知', '我國', '本', '聖', '人', '之', '都', '耳']

In [17]:
#실제로 잘 나옴
df_sent.loc[inword]

,ID,원문,번역문,sentence,token
22669,ITKC_MO_0004A_0050_010_0010,世多說東明王神異之事。雖愚夫騃婦。亦頗能說其事。僕嘗聞之。笑曰。先師仲尼。不語怪力亂神。此實...,\n세상에서 동명왕(東明王)의 신통하고 이상한 일을 많이 말한다. 비록 어리석은 남...,欲使夫天下知我國本聖人之都耳,"[欲, 使, 夫, 天下, 知, 我國, 本, 聖, 人, 之, 都, 耳]"
1200513,ITKC_MO_0299A_0250_010_0110,臣等昨日竭誠陳啓。恭竢兪音。而竟下勿煩之敎。相顧色沮。竊忖殿下拒之之深也。臣等非欲強爲煩籲。...,\n신들은 어제 온 정성을 다해 진달하고서 윤허한다는 명이 내려지기를 공손하게 기다...,我國素以禮義見稱於天下,"[我國, 素, 以, 禮, 義, 見, 稱, 於, 天下, 下]"
1421620,ITKC_MO_0329A_0060_010_0110,\n伏以君君臣臣父父子子。天地之常經也。利則就之。害則避之。人物之私心也。君之所以仁。臣之所...,\n삼가 아룁니다. 임금은 임금답고 신하는 신하답고 아비는 아비답고 자식은 자식다운...,則天下其謂我國何,"[則, 天下, 其, 謂, 我國, 何]"
1537458,ITKC_MO_0333A_0380_010_0360,我國東界一帶海水。無潮汐。故有東海無潮汐之說。而古之人未嘗道也。余嘗思其理而不可得。竊意非東...,"\n우리나라의 동해안 일대에는 밀물과 썰물 현상이 없기 때문에, 동해(東海)에는 으...",我國處天下之東北隅,"[我國, 處, 天下, 之, 東, 北, 隅]"
1964926,ITKC_MO_0392A_0070_010_0010,鼎重再拜言。近者得聞閭巷喧傳。北人發▦差自燕城。今十六上馬。廿六到我境。問其故。乃請兵事也。...,\n정중(鼎重)은 재배하고 말씀드립니다. 근래 여항(閭巷)에서 떠들썩하게 전해지는 ...,我國之負罪於天下久矣,"[我國, 之, 負, 罪, 於, 天下, 久, 矣]"
2252863,ITKC_MO_0402A_0070_060_0040,海之有潮汐。先儒說非一。谿谷斷以地之喘息之說爲是曰。地勢北爲背而南爲腹。腹有喘息而背無喘息。...,"\n바다에는 밀물과 썰물이 있는데, 선유(先儒)들의 주장은 한 가지가 아니다. 계곡...",我國處天下之東北隅,"[我國, 處, 天下, 之, 東, 北, 隅]"


### 원문 Word2vec

In [ ]:
# vector : 차원, window : 앞뒤 단어 보는 크기, min_count : 분석에 포함할 최소 단어 빈도수, sg : 0이면 cbow, 1이면 skip-gram
v300_w5_min5000_sg = Word2Vec(sentences=df_sent['token'].to_list(), vector_size=300, window=5, min_count=300,sg=1,seed=2021, workers=1)
v300_w5_min5000_sg.save('ch_v300_w5_min300_sg.model') # 모델 저장. 모델 로드할때는 model= Word2Vec.load('v200_w3_min300_sg.model')

In [ ]:
v300_w5_min300_sg = Word2Vec.load('ch_v300_w5_min300_sg.model')

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('國', topn=20)  # get other similar words

[('邦', 0.49060723185539246),
 ('廷', 0.3014860451221466),
 ('朝', 0.29671192169189453),
 ('域', 0.2852965295314789),
 ('家', 0.28310567140579224),
 ('麗', 0.2802332639694214),
 ('疆', 0.27800172567367554),
 ('狄', 0.276493638753891),
 ('貴', 0.2671152353286743),
 ('大', 0.2667595446109772),
 ('搢', 0.2611580193042755),
 ('億', 0.25519484281539917),
 ('也', 0.2522425651550293),
 ('者', 0.25005918741226196),
 ('王', 0.24942584335803986),
 ('閥', 0.24924707412719727),
 ('寇', 0.24853019416332245),
 ('社', 0.2478121966123581),
 ('宗', 0.24748802185058594),
 ('捍', 0.24625685811042786)]

In [ ]:
# 해당 단어와 문맥이 비슷한 유사어 추출
v300_w5_min300_sg.wv.most_similar('我', topn=20)

[('吾', 0.4932482838630676),
 ('余', 0.40199732780456543),
 ('予', 0.32671672105789185),
 ('朕', 0.30729252099990845),
 ('鮮', 0.2977280020713806),
 ('敝', 0.2936617136001587),
 ('人', 0.2914992570877075),
 ('賴', 0.28828781843185425),
 ('王', 0.28543445467948914),
 ('瀷', 0.28015753626823425),
 ('祖', 0.2762998342514038),
 ('皇', 0.2755458652973175),
 ('爾', 0.26780253648757935),
 ('矧', 0.2653608024120331),
 ('彼', 0.26424384117126465),
 ('汝', 0.2619032561779022),
 ('溘', 0.2569100856781006),
 ('寡', 0.25523895025253296),
 ('惟', 0.25349894165992737),
 ('饗', 0.25345274806022644)]

In [ ]:
v300_w5_min300_sg.wv.most_similar('吾', topn=20)

[('我', 0.49324825406074524),
 ('余', 0.48607683181762695),
 ('渠', 0.44041797518730164),
 ('汝', 0.41565221548080444),
 ('兄', 0.4037293493747711),
 ('瀷', 0.3959735631942749),
 ('弟', 0.3628588914871216),
 ('儕', 0.3415936529636383),
 ('其', 0.3364463746547699),
 ('姪', 0.32650044560432434),
 ('此', 0.321274071931839),
 ('爲', 0.3208879828453064),
 ('謂', 0.3201736807823181),
 ('予', 0.31980982422828674),
 ('君', 0.3190440237522125),
 ('則', 0.3160014748573303),
 ('但', 0.31013214588165283),
 ('生', 0.3090127110481262),
 ('人', 0.30574333667755127),
 ('父', 0.2988203465938568)]

In [ ]:
v300_w5_min300_sg.wv.most_similar('東', topn=20)

[('西', 0.5984089374542236),
 ('南', 0.5458069443702698),
 ('北', 0.4687977731227875),
 ('湖', 0.34696611762046814),
 ('瀛', 0.3141981065273285),
 ('海', 0.31271299719810486),
 ('球', 0.3096626102924347),
 ('萊', 0.3034631013870239),
 ('坡', 0.3008774220943451),
 ('淮', 0.30057740211486816),
 ('伽', 0.2989337146282196),
 ('江', 0.2839382290840149),
 ('山', 0.28217917680740356),
 ('邦', 0.2709294557571411),
 ('滄', 0.27080878615379333),
 ('迤', 0.26823994517326355),
 ('璿', 0.26807478070259094),
 ('嵩', 0.2678627371788025),
 ('州', 0.2661154866218567),
 ('嶠', 0.26283448934555054)]

In [ ]:
target_list = ['我', '吾', '東', '國']

In [ ]:
res = pd.DataFrame()
for target in target_list:
    res = pd.concat([res,pd.DataFrame(v300_w5_min300_sg.wv.most_similar(target, topn=20),columns=[target,'similarity'])],axis=1)  # get other similar words
res

,我,similarity,吾,similarity,東,similarity,國,similarity
0,吾,0.493248,我,0.493248,西,0.598409,邦,0.490607
1,余,0.401997,余,0.486077,南,0.545807,廷,0.301486
2,予,0.326717,渠,0.440418,北,0.468798,朝,0.296712
3,朕,0.307293,汝,0.415652,湖,0.346966,域,0.285297
4,鮮,0.297728,兄,0.403729,瀛,0.314198,家,0.283106
5,敝,0.293662,瀷,0.395974,海,0.312713,麗,0.280233
6,人,0.291499,弟,0.362859,球,0.309663,疆,0.278002
7,賴,0.288288,儕,0.341594,萊,0.303463,狄,0.276494
8,王,0.285434,其,0.336446,坡,0.300877,貴,0.267115
9,瀷,0.280158,姪,0.326500,淮,0.300577,大,0.266760


In [ ]:
res.to_excel('한국고전종합DB_원문_word2vec.xlsx')